In [1]:
import gensim
import os
import collections
import smart_open
import random
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint

d:\Anaconda3\envs\MLOntology\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#global variabls

directory_path = "D:/MLOntology/NCIt/"
data_path = directory_path + "data/"
vector_model_path = directory_path +"vectorModel/"
cnn_model_path = directory_path +"cnnModel/"

In [3]:
import re
def get_trailing_number(s):
    m = re.search(r'\d+$', s)
    return m.group() if m else None

In [4]:
#read class label file
#create mapping from id to labels  
#iso-8859-1 , encoding="iso-8859-1"
conceptLabelDict={}
errors=[]

def read_label(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                conceptID = get_trailing_number(splitted[1])
                conceptLabelDict[conceptID] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

label_file = data_path + "ontClassLabels_owl_ncit.txt"
read_label(label_file)
print(conceptLabelDict["4863"])
print(conceptLabelDict["115117"])
print(errors)


prostate carcinoma
stage ia esophageal cancer ajcc v7
[]


In [5]:
print(len(conceptLabelDict))

9955


In [6]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t", 2)
            
            line = splitted[2]
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                tagID = get_trailing_number(splitted[0])
                conceptMappingDict[i]= int(tagID)
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [tagID])

In [7]:
conceptMappingDict={}

train_file = data_path + "ontClassTopology_owl_ncit.txt"

train_corpus = list(read_corpus(train_file))

In [8]:
print(len(train_corpus))
train_corpus[1296:1299]

9955


[TaggedDocument(words=['neoplasm', 'by', 'special', 'category', 'neoplasm', 'by', 'obsolete', 'classification', 'hematopoietic', 'and', 'lymphoid', 'neoplasm', 'antiquated', 'lymphoma', 'by', 'working', 'formulation', 'antiquated', 'intermediate', 'grade', 'lymphoma', 'adult', 'non', 'hodgkin', 'lymphoma', 'grade', 'intermediate', 'grade', 'adult', 'non', 'hodgkin', 'lymphoma', 'stage', 'iii', 'intermediate', 'grade', 'adult', 'non', 'hodgkin', 'lymphoma', 'stage', 'iii', 'intermediate', 'grade', 'adult', 'non', 'hodgkin', 'lymphoma', 'stage', 'iii', 'intermediate', 'grade', 'adult', 'non', 'hodgkin', 'lymphoma'], tags=['8131']),
 TaggedDocument(words=['neoplasm', 'by', 'morphology', 'neoplasm', 'by', 'site', 'epithelial', 'neoplasm', 'head', 'and', 'neck', 'neoplasm', 'carcinoma', 'neck', 'neoplasm', 'malignant', 'head', 'and', 'neck', 'neoplasm', 'mucin', 'producing', 'carcinoma', 'laryngeal', 'neoplasm', 'malignant', 'neck', 'neoplasm', 'head', 'and', 'neck', 'carcinoma', 'mucoepide

In [15]:
cores = multiprocessing.cpu_count()

print(cores)
models = [
    # PV-DBOW 
    Doc2Vec(dm=0, dbow_words=1, vector_size=128, window=20, min_count=1, epochs=10, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, vector_size=128, window=20, min_count=1, epochs=10, workers=cores),
]


4


In [16]:
models[0].build_vocab(train_corpus)
print(str(models[0]))
models[1].reset_from(models[0])
print(str(models[1]))

Doc2Vec(dbow+w,d128,n5,w20,s0.001,t4)
Doc2Vec(dm/m,d128,n5,w20,s0.001,t4)


In [17]:
for model in models:
    %%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

Wall time: 31 s
Wall time: 6.7 s


In [18]:
for i, model in enumerate(models):
    temp_path = vector_model_path + "model" + str(i)  # creates a temp file
    model.save(temp_path)

In [19]:
for model in models:
    print(str(model))
    inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
    pprint(model.docvecs.most_similar([inferred_vector], topn=20))

Doc2Vec(dbow+w,d128,n5,w20,s0.001,t4)
[('4380', 0.6904844045639038),
 ('3615', 0.6812392473220825),
 ('45750', 0.6809202432632446),
 ('4453', 0.6776319742202759),
 ('45749', 0.6672717332839966),
 ('7744', 0.6627799272537231),
 ('116899', 0.6626230478286743),
 ('45752', 0.6624276041984558),
 ('4494', 0.6618131995201111),
 ('3841', 0.6600770950317383),
 ('6387', 0.6590962409973145),
 ('8539', 0.6583346724510193),
 ('3086', 0.6558387279510498),
 ('7053', 0.6551984548568726),
 ('3205', 0.654028594493866),
 ('27127', 0.6531809568405151),
 ('45751', 0.6502649188041687),
 ('7393', 0.649591326713562),
 ('4487', 0.6492371559143066),
 ('3480', 0.6482855081558228)]
Doc2Vec(dm/m,d128,n5,w20,s0.001,t4)
[('142823', 0.6021954417228699),
 ('6587', 0.45289045572280884),
 ('6721', 0.44645121693611145),
 ('3263', 0.4138094186782837),
 ('3084', 0.4113937318325043),
 ('4370', 0.4108443260192871),
 ('7576', 0.4001743793487549),
 ('4228', 0.3985741138458252),
 ('139533', 0.38129493594169617),
 ('90513', 0.37

In [ ]:
model.docvecs['133521']

In [ ]:
#gensim.utils.simple_preprocess??
gensim.models.doc2vec.TaggedDocument??

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=1, iter=5, alpha=0.025, min_alpha=0.025, sample = 1e-3)

gensim.models.doc2vec.Doc2Vec??

model.build_vocab(train_corpus)

len(model.wv.vocab)

%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

# inferred_vector = model.infer_vector(['clinical', 'finding', 'evaluation', 'prevent', 'sampling', 'foot'])
inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])


sims = model.docvecs.most_similar([inferred_vector], topn=10)
pprint(sims)

for sim in sims:
    print(train_corpus[sim[0]].words, "score: ", sim[1])
    
for sim in sims:
    print(conceptMappingDict[sim[0]], "score: ", sim[1])
  

from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file

model.save(temp_path)  # save the model

new_model = gensim.models.Doc2Vec.load(temp_path)